<a href="https://colab.research.google.com/github/otabek5454/net_intrusion_detection/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
!unzip MachineLearningCSV.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
To: /content/MachineLearningCSV.zip
100% 235M/235M [00:04<00:00, 53.3MB/s]
Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  i

In [ ]:
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/models.py
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py

--2023-07-29 09:43:13--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15128 (15K) [text/plain]
Saving to: ‘models.py’

models.py           100%[===================>]  14.77K  --.-KB/s    in 0s      

2023-07-29 09:43:13 (143 MB/s) - ‘models.py’ saved [15128/15128]

--2023-07-29 09:43:13--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790 (3.7K) [text/p

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from preprocessing import load_data, balance_data, normalize
from models import Classifier
import time
from os.path import join
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

In [ ]:
dataroot = 'MachineLearningCVE/'

from preprocessing import read_data
data = read_data(dataroot,'*.pcap_ISCX.csv')

# Load and preprocess the data
X, y = load_data(dataroot)

[########################################] | 100% Completed | 28.19 s
[########################################] | 100% Completed | 27.31 s
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


In [ ]:
X, y = balance_data(X, y, seed=42)
X = normalize(X)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and remaining sets first
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Convert the data into PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Create PyTorch data loaders
batch_size = 32
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

device = "cuda"

In [ ]:
#hyper-parameters
hyper_params = {
    'batch_size': 5120,
    'optim': 'Adam',
    'learning_rates': [1e-4, 1e-2, 1e-0],
    'regularizations': [1e-6, 1e-4, 1e-2],
    'num_layers': 3,
    'method': 'softmax',
    'num_epochs': 20
}

# Determine input dimensions and number of classes
input_dim = X_train.shape[1]
num_class = len(np.unique(y_train))

# Initialize tracking variables
best_model, best_acc = None, -1
accuracies = {}

In [ ]:
def getClassifier(args, runs_dir=None):

    (method, optim, lr, reg, batch_size, input_dim, num_class, num_epochs) = args
    if runs_dir is not None:
        ensure_dir(runs_dir)

    clf = Classifier(method, input_dim, num_class, lr=lr, reg=reg, num_epochs=num_epochs,
                        batch_size=batch_size, runs_dir=runs_dir)

    return clf

    return clf
def ensure_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
# Setting up device
device = "cuda"


In [ ]:
# Convert data types
X_train = X_train.float()
y_train = y_train.long()

# Move data to the device
X_train = X_train.to(device)
y_train = y_train.to(device)


In [ ]:
# Convert the data into numpy arrays
X_train, y_train = X_train.cpu().numpy(), y_train.cpu().numpy()
X_val, y_val = X_val.cpu().numpy(), y_val.cpu().numpy()
X_test, y_test = X_test.cpu().numpy(), y_test.cpu().numpy()


In [ ]:
# Grid search over learning rates and regularization strengths
for lr in hyper_params['learning_rates']:
    for reg in hyper_params['regularizations']:
        # Create model configuration
        config =  f"{hyper_params['method']}/train/optim_{hyper_params['num_layers']}_{hyper_params['optim']}_lr_{lr}_reg_{reg}_bs_{hyper_params['batch_size']}"
        runs_dir = join(dataroot, 'runs', config)

        # Initialize classifier with current parameters
        classifier_args = (hyper_params['method'], hyper_params['optim'], lr, reg, hyper_params['batch_size'], input_dim, num_class, 5)  # Updated to 5 epochs
        clf = getClassifier(classifier_args, runs_dir)

        # Train and evaluate model
        start_time = time.time()
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        acc = metrics.balanced_accuracy_score(y_test, predictions)

        # Update best model if current model is better
        if acc > best_acc:
            best_model = clf
            best_acc = acc

        # Store accuracy for current hyper-parameters
        accuracies[(lr, reg)] = acc

        end_time = time.time()
        print(f"Model trained in {end_time - start_time:.0f} sec")

# Display accuracies
print("Accuracies for different hyper-parameters:")
for params, acc in accuracies.items():
    print(f"LR: {params[0]}, Reg: {params[1]}, Acc: {acc}")


Loaded MachineLearningCVE/runs/softmax/train/optim_3_Adam_lr_0.0001_reg_1e-06_bs_5120 model trained with batch_size = 5120, seen 19 epochs and 368 mini batches
best epoch 20, best batch 369
bst acc  62.696199056158164
Model trained in 17 sec
Loaded MachineLearningCVE/runs/softmax/train/optim_3_Adam_lr_0.0001_reg_0.0001_bs_5120 model trained with batch_size = 5120, seen 2 epochs and 1 mini batches
best epoch 3, best batch 2
bst acc  42.01541034786436
Epoch [4/5], Step [3/348], Loss: 2.4347
Epoch [4/5], Step [53/348], Loss: 2.4080
Epoch [4/5], Step [103/348], Loss: 2.4063
Epoch [4/5], Step [153/348], Loss: 2.3779
Epoch [4/5], Step [203/348], Loss: 2.3659
Epoch [4/5], Step [253/348], Loss: 2.3522
Epoch [4/5], Step [303/348], Loss: 2.3252
Epoch [5/5], Step [4/348], Loss: 2.3178
Epoch [5/5], Step [54/348], Loss: 2.3084
Epoch [5/5], Step [104/348], Loss: 2.2945
Epoch [5/5], Step [154/348], Loss: 2.2791
Epoch [5/5], Step [204/348], Loss: 2.2652
Epoch [5/5], Step [254/348], Loss: 2.2483
Epoch 